In [70]:
import numpy as np
from collections import OrderedDict
import math

from estimates_calculations import Concept#, likelihood, posteriors, 
from concepts_database import gen_base_concepts

In [77]:
base_concepts = [Concept('power_of_2', 
                         lambda x: 2 ** int(math.log(x,2)) == x, 
                         0.2),
                 Concept('even',
                         lambda x: x%2 == 0,
                         0.2),
                 Concept('odd',
                         lambda x: x%2 == 1,
                         0.2),
                 Concept('multiples_of_3',
                         lambda x: x%3==0,
                         0.1),
                 Concept('multiples_of_10',
                         lambda x: x%10==0,
                         0.1)
                ]

Concept.normalize_priors(base_concepts)
for c in base_concepts: print(c)

Name: power_of_2,         	Prior: 0.25,	Size: 511
Name: even,               	Prior: 0.25,	Size: 62750
Name: odd,                	Prior: 0.25,	Size: 62500
Name: multiples_of_3,     	Prior: 0.12,	Size: 41583
Name: multiples_of_10,    	Prior: 0.12,	Size: 12750


In [71]:
def likelihood(data, hypothesis):
    if all([hypothesis.checker(x) for x in data]):
        return (1/hypothesis.size)**len(data)
    else:
        return 0

In [72]:
def posteriors(data, hypotheses):
    denominator = 0
    numerators = dict()
    for h in hypotheses:
        numerators[h.name] = likelihood(data, h) * h.prior
        denominator += numerators[h.name]
        
    for k in numerators:
        numerators[k] /= denominator
        
    return numerators

In [73]:
def number_game(data, concepts):
    results = posteriors(data, concepts)
    return OrderedDict(sorted(results.items(), key=lambda t: -t[1]))

In [78]:
number_game([3,9,15], base_concepts)

OrderedDict([('multiples_of_3', 0.6293147478415455),
             ('odd', 0.37068525215845455),
             ('power_of_2', 0.0),
             ('even', 0.0),
             ('multiples_of_10', 0.0)])

In [79]:
def MAP(data, concepts):
    eps = 1e-9
    func = lambda data, c: np.log(likelihood(data, c)+eps)+np.log(c.prior+eps) 
    log_posteriori = np.asarray([func(data, c) for c in concepts])
    return concepts[np.argmax(log_posteriori)]

In [80]:
print(MAP([3,9,15], base_concepts))

Name: odd,                	Prior: 0.25,	Size: 62500


In [81]:
def MLE(data, concepts):
    eps = 1e-9
    func = lambda data, c: np.log(likelihood(data, c)+eps)
    log_likelihood = np.asarray([func(data, c) for c in concepts])
    return concepts[np.argmax(log_likelihood)]

In [82]:
print(MLE([3,9,15], base_concepts))

Name: multiples_of_3,     	Prior: 0.12,	Size: 41583
